# Naïve Bayes

## Vi trenger en klassifikator som kan lagre det den lærer

In [2]:
class classifier:
    def __init__(self, getfeatures, filename=None):
        self.fc = {}
        self.cc = {}
        self.getfeatures = getfeatures

fc lagrer antall ganger en feature forekommer i en gitt klasse, og cc lagrer hvor mange ganger en klasse har blitt brukt.

Dersom klassifikatoren trenes på setningen "i liked the Da Vinci Code a lot" med klassen "positive", vil fc bli `{'i': {'positive': 1, 'negative': 0}, 'liked': {'positive': 1, 'negative': 0}, 'the': {'positive': 1, 'negative': 0}, 'Da': {'positive': 1, 'negative': 0}, ...}`

Vi legger til følgende hjelpemetoder for å oppdatere/lese fc og cc:

In [3]:
        # Øk feature-counten for en klasse
    def incf(self, f, cat):
        self.fc.setdefault(f, {})
        self.fc[f].setdefault(cat, 0)
        self.fc[f][cat] += 1


        # Øk counten for en klasse/kategori
    def incc(self, cat):
        self.cc.setdefault(cat, 0)
        self.cc[cat] += 1


        # Feature-count i en kategori
    def fcount(self, f, cat):
        if f in self.fc and cat in self.fc[f]:
            return float(self.fc[f][cat])
        return 0

        # Antall elementer i en kategori
    def catcount(self, cat):
        if cat in self.cc:
            return float(self.cc[cat])
        return 0

        # Antall elementer
    def totalcount(self):
        return sum(self.cc.values())

        # Alle kategorier
    def categories(self):
        return self.cc.keys()

IndentationError: unexpected indent (<ipython-input-3-79fee6e93c8e>, line 2)

### Trening

`train` tar inn et dokument og den tilhørende klassen, og oppdaterer `fc` og `cc` i klassifikatoren i henhold til dataen.

In [ ]:
    def train(self, item, cat):
        features = self.getfeatures(item)

        for f in features:
            self.incf(f, cat)

        self.incc(cat)

Nå har vi en klassifikator med en treningsfunksjon som holder styr på hvor mange ganger en `feature` forekommer i hver `klasse/kategori`.

## Sannsynlighet

Vi beregner `P (f|c)`, altså antall ganger denne `featuren` har forekommet i klassen delt på antall elementer i klassen.

In [ ]:
    def fprob(self, f, cat):
        if self.catcount(cat) == 0:
            return 0

        return self.fcount(f, cat)/self.catcount(cat)

Dersom et ord først havner i klassen `positiv`, vil `fprob` plutselig gi en sjanse på 0 for at dette ordet vil havne i klassen `negativ` senere. For å komme seg rundt dette problemet, velger vi en `prior probability`, der vi setter en foreløpig sannsynlighet for klassene. Vi lager en funksjon for å vekte det.

In [ ]:
    def weightedprob(self, f, cat, prf, weight=1.0, ap=0.5):
        # Vi velger 0.5 som prior probability
        
        # Kalkuler sannsynlighet
        basicprob = prf(f, cat)

        # Tell antall ganger denne featuren har forekommet i kategoriene
        totals = sum([self.fcount(f, c) for c in self.categories()])

        # Kalkuler det vektede snittet
        bp = ((weight * ap) + (totals * basicprob)) / (weight + totals)
        return bp

## naïve Bayes-klassifikator

Nå skal vi bruke naïve Bayes for å klassifisere dokumenter. Vi lager først en funksjon for å regne ut P (dokument|klasse), og siden vi antar at featurene er uavhengige, er dette det samme som P (ord1|klasse) * P (ord2|klasse) * ...

Vi lager en subklasse av `classifier`, `naivebayes`:

In [ ]:
class naivebayes(classifier):
    def docprob(self, item, cat):
        features = self.getfeatures(item)
        
        # Ganger sammen sannsynlighetene for alle featurene/ordene
        p = 1
        for f in features:
            p *= self.weightedprob(f, cat, self.fprob)
            return p

Vi kan nå beregne `P(dokument|klasse)`, men for å klassifisere nye dokumenter trenger vi `P(klasse|dokument`. Heldigvis hjelper Bayes´ teorem oss her:

`P(klasse|dokument) = (P(dokument|klasse) * P(klasse)) / P(dokument)`

Og siden vi deler på `P(dokument)` for alle klassene, kan vi fjerne denne. Vi beregner `P(klasse|dokument)` slik:

In [ ]:
    def prob(self, item, cat):
        catprob = self.catcount(cat) / self.totalcount()
        docprob = self.docprob(item, cat)
        return docprob * catprob

### Classify

Det eneste vi mangler nå, er å beregne `P(klasse|dokument)` for hver klasse, så finne vi ut hvor dokumentet hører hjemme.

In [ ]:
    def classify(self, document):
        probs = {}

        # Finn klassen med høyest sannsynlighet
        max = 0.0

        for cat in self.categories():
            probs[cat] = self.prob(document, cat)
            if probs[cat] > max:
                max = probs[cat]
                best = cat

        return best

## Les inn data

Hver linje i treningsfilen inneholder et tall (1 eller 0) og en setning. 1 indikerer at setningen er positiv og 0 indikerer at setningen er negativ. Her er et par eksempler fra filen:

```
1	Hey I loved The Da Vinci Code!..
1	Mission Impossible III was awesome.
0	I hate Harry Potter, it's retarted, gay and stupid and there's only one Black guy...

```

Les inn treningsdataen og klassifiser nye dokumenter slik:

In [1]:
def retrieve_training_data(_file):
    with open(_file) as f:
        content = f.readlines()

    data = [line.strip().split('\t') for line in content]

    return(data)

def train_from_file(classifier, _file="training.txt"):
    data = retrieve_training_data(_file)

    for line in data:
        classifier.train(line[1], 'positive' if line[0] == '1' else 'negative')

Så kan du trene nye dokumenter slik:

In [ ]:
classifier = naivebayes(getwords)
train_from_file(classifier)

print(classifier.classify('i love harry potter'))
print(classifier.classify('i hate trump'))

## Oppgaver

1. Istedenfor å trene på hele filen, bruk 80% av den på trening og 20% av den til testing. Hvor høy precision og recall får du?

2. Kan du finne en måte å håndtere negering på? Hvordan påvirker det precision og recall?